In [1]:
%load_ext autoreload
%autoreload 2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import seaborn as sns
import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh

from Loss import ber_loss

from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps
from lava.proc import io
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import RefPort

from lava.lib.dl import netx

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation


%load_ext autoreload

%autoreload 2

torch.__version__
# The coarse network structure is dicated by the Fashion MNIST dataset. 
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:1')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")
    
print(device)
tau_mem = 10e-3
tau_syn = 5e-3
time_step = 1e-3
alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

print("init done")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu
init done


In [2]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [55]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]
i = 1
rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
        N_reservoir=16,
        spectral_radius=0.2,
        sparsity=0.4,
        noise=1e-6,
        lut_activation=False,  # True,
        tanh_lut=tanh_lut,
        input_scale=25,  #40, #50, # 25,
        reservoir_input_scale = 8,  #4,  #5,
        show_wout=False,
        output_folder= output_folder,
        debug=debug,
        use_fpga= None,
        sock= None,  # usock
        addr = None) # addr

train_input, train_label, test_input, test_label = rc.run()
RC_test_input = np.load('gt_test_input_1.npy')
RC_train_input = np.load('gt_train_input_1.npy')
RC_train_label = np.load('gt_train_label_1.npy')

print(RC_test_input.shape, test_input.shape)
print("test_input_diff: ", torch.nn.MSELoss()(torch.tensor(RC_test_input), torch.tensor(test_input)))

print("train_input_diff: ", torch.nn.MSELoss()(torch.tensor(RC_train_input), torch.tensor(train_input)))

print("train_label_diff: ", torch.nn.MSELoss()(torch.tensor(RC_train_label), torch.tensor(train_label)))

train_mean = np.mean(train_input)
train_std = np.std(train_input)

train_input = (train_input - train_mean) / train_std
test_input = (test_input - train_mean) / train_std
train_label = 1.0 * train_label

(7521, 4) (7521, 4)
test_input_diff:  tensor(4.4101e-10, dtype=torch.float64)
train_input_diff:  tensor(4.8907e-11, dtype=torch.float64)
train_label_diff:  tensor(0., dtype=torch.float64)


In [3]:
print(train_input.shape)
print(test_input.shape)
print(train_label.shape)
print(test_label.shape)

(7521, 4)
(7521, 4)
(7521, 2)
(89, 64)


In [4]:
import pandas as pd


def pre_processing(train_input, train_label):
    idx_p = 10
    begin = 0 # N_total_frame * N_sync_frame
    
    # label index for data
    train_input_df = pd.DataFrame(train_input, columns = ['1','2', '3', '4'])
    #train_input_df['L1_idx'] = train_input_df.index % idx_p

    # label index for label
    train_label_df = pd.DataFrame(train_label, columns = ['L1','L2'])
    train_label_df['L1_idx'] = train_label_df.index % idx_p
    
    # split training and testing data
    test_input_df, test_label_df = train_input_df.iloc[75* 80 + 1:, :], train_label_df.iloc[75* 80 + 1:, :]
    train_input_df, train_label_df = train_input_df.iloc[:75* 80 + 1, :], train_label_df.iloc[:75* 80 + 1, :]

    # group by 
    #mapping = train_label_df.loc[begin:, :].groupby(by='L1_idx').mean().reset_index().loc[:, ['L1', 'L2', 'L1_idx']] 
    
    #train_input_df = pd.merge(train_input_df, mapping, how='left', on='L1_idx')

    #train_input_df = pd.get_dummies(train_input_df, prefix=['L'], columns=['L1_idx'])

    train_input_df = train_input_df.loc[begin:, :]


    print(train_input_df.head())
    
    # testing data
    # group by
    #test_input_df = test_input_df.merge(mapping, how = 'left', on='L1_idx')

    #test_input_df = pd.get_dummies(test_input_df, prefix=['L'], columns=['L1_idx'])

    print(test_input_df.head())

    train_input = train_input_df.to_numpy()
    test_input = test_input_df.to_numpy()
    
    train_label = train_label_df.drop(['L1_idx'], axis=1).to_numpy()
    test_label = test_label_df.drop(['L1_idx'], axis=1).to_numpy()
    
    print(train_input.shape)
    print(test_input.shape)
    
    
    return train_input, train_label, test_input, test_label

train_input, train_label, test_input, test_label = pre_processing(train_input, train_label)

          1         2         3         4
0  0.004563  0.004563 -1.888935  0.282201
1 -1.888935  0.282201  0.813650 -5.120430
2  0.813650 -5.120430  2.435637  1.351414
3  2.435637  1.351414  0.283448  5.122371
4  0.283448  5.122371  0.001910  3.788948
             1         2         3         4
6001 -0.660964  0.678438 -0.212950 -1.236282
6002 -0.212950 -1.236282 -0.928489 -0.692997
6003 -0.928489 -0.692997 -1.297116  0.326895
6004 -1.297116  0.326895  0.259780 -0.845012
6005  0.259780 -0.845012  0.519246  0.137295
(6001, 4)
(1520, 4)


In [5]:
nb_inputs  = 6
nb_steps  = 100

batch_size = 128

import scipy.io

from dataset import Dataset, RegTorchSeasonalitySpikingDataset, RegSpikingDataset, RegTorchSpikingDataset, RegTorchSeasonalityLinearSpikingDataset
train_data = RegTorchSeasonalityLinearSpikingDataset(train_input, train_label, nb_inputs, nb_steps, on_loihi=True)
test_data = RegTorchSeasonalityLinearSpikingDataset(test_input, test_label, nb_inputs, nb_steps, on_loihi=True)
print(len(train_data))
print(train_data[0][0].shape)
print(train_data[0][1].shape)

6001
(24, 100)
(2,)


In [6]:
net = netx.hdf5.Network(net_config='net_snn.net')
print(net)

|   Type   |  W  |  H  |  C  | ker | str | pad | dil | grp |delay|
|Dense     |    1|    1|    8|     |     |     |     |     |True |
|Dense     |    1|    1|    1|     |     |     |     |     |True |


In [7]:
print(f'There are {len(net)} layers in network:')

for l in net.layers:
    print(f'{l.block:5s} : {l.name:10s}, shape : {l.shape}')

There are 2 layers in network:
Dense : Process_1 , shape : (8,)
Dense : Process_4 , shape : (1,)


In [8]:
from lava.proc import io

# Instantiate the processes
dataloader = io.dataloader.SpikeDataloader(dataset=test_data, interval=nb_steps)
output_logger = io.sink.RingBuffer(shape=net.out_layer.shape, buffer=nb_steps)
#gt_logger = io.sink.RingBuffer(shape=(2,), buffer=nb_steps)
# reset
for i, l in enumerate(net.layers[:-1]):
    u_resetter = io.reset.Reset(interval=nb_steps, offset=i)
    v_resetter = io.reset.Reset(interval=nb_steps, offset=i)
    u_resetter.connect_var(l.neuron.u)
    v_resetter.connect_var(l.neuron.v)

# Connect the input to the network:
#dataloader.ground_truth.connect(gt_logger.a_in)
dataloader.s_out.connect(net.inp)

# Connect network-output to the output process
net.out.connect(output_logger.a_in)

In [15]:
pretrained_model = torch.load('./net_dnn.pth', map_location=torch.device('cpu'))
class DNNNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 2)
        self.fc2 = nn.Linear(16 + 1 + 24, 8)
        self.fc3 = nn.Linear(16, 2)
        #self.fc3 = nn.Linear(128, 128)
        #self.fc4 = nn.Linear(128, 2)
        self.act = nn.ReLU()

    def forward(self,x, x1, x2):
        x = x.flatten(start_dim=1)
        x = torch.cat((x, x2), axis=1)
        x = self.fc1(x)
        return x
    
net_dnn = DNNNetwork(1 * 100 + nb_inputs * 4 + 0 + 0 * nb_inputs, 2).to(device)
net_dnn.load_state_dict(pretrained_model)

<All keys matched successfully>

In [17]:
class CustomRunConfig(Loihi1SimCfg):
    def select(self, proc, proc_models):
        # customize run config to always use float model for io.sink.RingBuffer
        if isinstance(proc, io.sink.RingBuffer):
            return io.sink.PyReceiveModelFloat
        else:
            return super().select(proc, proc_models)

In [18]:
test_data_no_loihi = RegTorchSeasonalityLinearSpikingDataset(test_input, test_label, nb_inputs, nb_steps, on_loihi=False)

In [19]:
error = 0.0
size = 20
all_outputs = []
targets = []
for i in range(len(test_data_no_loihi)):
    print(i)
    target = test_data[i][1]
    input, input1, input2, target = test_data_no_loihi[i]
    targets.append(np.expand_dims(target, axis=0))
    
    input1 = torch.tensor(input1, dtype=torch.float).unsqueeze(0).to(device)
    input2 = torch.tensor(input2, dtype=torch.float).unsqueeze(0).to(device)
    
    run_condition = RunSteps(num_steps=nb_steps)
    run_config = Loihi1SimCfg(select_tag='fixed_pt', select_sub_proc_model=True)
    net.run(condition=run_condition, run_cfg=run_config) 
    output = output_logger.data.get()
    output = torch.Tensor(output).to(device)
    output = net_dnn(output, input1, input2).cpu().detach().numpy()
    all_outputs.append(output)

all_output = np.concatenate(all_outputs, axis=0)
targets = np.concatenate(targets, axis=0)

print(all_output.shape, targets.shape)

predict_time = rc.time_to_freq(all_output, N_total_frame=19, remove_delay=False)
target_time = rc.time_to_freq(targets, N_total_frame=19, remove_delay=False)
print("testing ber: ", 0.5*rc.my_new_test(predict_time, target_time))

net.stop()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
def test(test_loader, net_snn, net_dnn, rc, num_frame = 19):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target.cpu().detach().numpy())
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
    
        output = net_snn(input)
        
        output = net_dnn(output, input1, input2).cpu().detach().numpy()
        all_output.append(output)
        
    
    all_output = np.concatenate(all_output, axis=0)
    labels = np.concatenate(labels, axis=0)
   
    print("testing MSE: ", torch.nn.MSELoss()(torch.tensor(all_output), torch.tensor(labels)))
    print("testing MSE 0: ", torch.nn.MSELoss()(torch.tensor(all_output[:, 0]), torch.tensor(labels[:, 0])))
    print("testing MSE 1: ", torch.nn.MSELoss()(torch.tensor(all_output[:, 1]), torch.tensor(labels[:, 1])))
    #print(all_output.shape, labels.shape)
    print("percent loss: ", np.mean(np.abs(all_output - labels) / (np.abs(labels) + 1e-6)))
    predict_time = rc.time_to_freq(all_output, num_frame, remove_delay=False)
    target_time = rc.time_to_freq(labels, num_frame, remove_delay=False)
    print("testing ber: ", rc.my_new_test(predict_time, target_time))

test(test_loader, net_snn, net_dnn, rc, num_frame=19)

NameError: name 'test_loader' is not defined